In [16]:
import pandas as pd
import requests
import json

In [54]:
loc = 'C:\\Users\\Darrick\\Documents\\Jupyter\\senior-project\\stats\\'
season = '2016-17'

# Basic Defensive Statistics

Gets basic defense stats from nba.com

In [55]:
headers = {
    'Accept': 'application/json, text/plain, */*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://www.nba.com/'
}

# Retrieve both defensive dashboard and defensive impact
r_dd = requests.get('https://stats.nba.com/stats/leaguedashptdefend?College=&Conference=&Country=&DateFrom=&DateTo=&DefenseCategory=Overall&Division=&DraftPick=&DraftYear=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&Season=' + season + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight=', headers=headers, timeout=10)
r_di = requests.get('https://stats.nba.com/stats/leaguedashptstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&PlayerExperience=&PlayerOrTeam=Player&PlayerPosition=&PtMeasureType=Defense&Season=' + season + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight=', headers=headers, timeout=10)

In [56]:
r_dd_dict = json.loads(r_dd.content)
r_di_dict = json.loads(r_di.content)

dd_df = pd.DataFrame(data=r_dd_dict['resultSets'][0]['rowSet'], columns=r_dd_dict['resultSets'][0]['headers'])
di_df = pd.DataFrame(data=r_di_dict['resultSets'][0]['rowSet'], columns=r_di_dict['resultSets'][0]['headers'])

dd_df.sort_values(by='PLAYER_NAME', inplace=True)
di_df.sort_values(by='PLAYER_NAME', inplace=True)

In [57]:
# Remove players not in both dataframes
diff = list(set(di_df['PLAYER_ID'].tolist()) - set(dd_df['CLOSE_DEF_PERSON_ID'].tolist()))

di_df = di_df[~di_df['PLAYER_ID'].isin(diff)]

# Combine dataframes
def_data = dd_df.join(di_df[di_df.columns[5:11]])

# Drop unnecessary columns
def_data.drop(columns=['PLAYER_LAST_TEAM_ID', 'PLAYER_LAST_TEAM_ABBREVIATION', 'PLAYER_POSITION', 'FREQ'], inplace=True)
def_data.rename(columns={'CLOSE_DEF_PERSON_ID' : 'PLAYER_ID'}, inplace=True)

def_data.head()

,PLAYER_ID,PLAYER_NAME,AGE,GP,G,D_FGM,D_FGA,D_FG_PCT,NORMAL_FG_PCT,PCT_PLUSMINUS,W,L,MIN,STL,BLK,DREB
423,1627773,AJ Hammons,24.0,17,17,1.76,3.71,0.476,0.481,-0.004,27.0,27.0,14.8,0.3,0.6,2.8
375,201166,Aaron Brooks,32.0,61,61,2.21,4.80,0.461,0.434,0.027,4.0,5.0,9.4,0.2,0.0,0.9
141,203932,Aaron Gordon,21.0,80,80,3.93,8.99,0.437,0.455,-0.019,1.0,1.0,11.6,0.0,0.0,1.5
481,1626151,Aaron Harrison,22.0,4,4,0.75,1.25,0.600,0.304,0.296,34.0,36.0,30.9,0.7,0.4,5.0
441,203940,Adreian Payne,26.0,17,17,1.47,3.35,0.439,0.449,-0.010,1.0,7.0,9.5,0.1,0.1,1.8


# Hustle Statistics

Gets contested shots (2p and 3p), deflections, charges drawn, defensive boxouts, %boxout

In [58]:
r = requests.get('https://stats.nba.com/stats/leaguehustlestatsplayer?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=' + season + '&SeasonSegment=&SeasonType=Regular+Season&TeamID=0&VsConference=&VsDivision=&Weight=', headers=headers, timeout=10)

In [59]:
r_dict = json.loads(r.content)

df = pd.DataFrame(data=r_dict['resultSets'][0]['rowSet'], columns=r_dict['resultSets'][0]['headers'])
df.drop(columns=['TEAM_ID', 'TEAM_ABBREVIATION', 'AGE', 'G', 'MIN',
                'SCREEN_AST_PTS', 'SCREEN_ASSISTS', 'OFF_LOOSE_BALLS_RECOVERED', 'DEF_LOOSE_BALLS_RECOVERED',
                'LOOSE_BALLS_RECOVERED', 'PCT_LOOSE_BALLS_RECOVERED_OFF', 'PCT_LOOSE_BALLS_RECOVERED_DEF',
                'OFF_BOXOUTS', 'BOX_OUTS', 'BOX_OUT_PLAYER_TEAM_REBS', 'BOX_OUT_PLAYER_REBS', 'PCT_BOX_OUTS_OFF',
                'PCT_BOX_OUTS_DEF', 'PCT_BOX_OUTS_TEAM_REB'], inplace=True)

# Remove players not in df
diff = list(set(df['PLAYER_ID'].tolist()) - set(def_data['PLAYER_ID'].tolist()))
df = df[~df['PLAYER_ID'].isin(diff)]
df.sort_values(by='PLAYER_NAME', inplace=True)

# Insert hustle data
def_data = def_data.join(df[df.columns[2:]])

# Set season ID
def_data['SEASON_ID'] = '220' + season[2:4]

def_data.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'AGE', 'GP', 'G', 'D_FGM', 'D_FGA',
       'D_FG_PCT', 'NORMAL_FG_PCT', 'PCT_PLUSMINUS', 'W', 'L', 'MIN', 'STL',
       'BLK', 'DREB', 'CONTESTED_SHOTS', 'CONTESTED_SHOTS_2PT',
       'CONTESTED_SHOTS_3PT', 'DEFLECTIONS', 'CHARGES_DRAWN', 'DEF_BOXOUTS',
       'PCT_BOX_OUTS_REB', 'SEASON_ID'],
      dtype='object')

# Defensive Ratings

In [60]:
r = requests.get('https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Defense&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=' + season + '&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=&Weight=', headers=headers, timeout=10)

In [61]:
r_dict = json.loads(r.content)

df = pd.DataFrame(data=r_dict['resultSets'][0]['rowSet'], columns=r_dict['resultSets'][0]['headers'])
df.drop(columns=['NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'DREB', 'PCT_DREB', 'STL',
        'BLK', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'DEF_RATING_RANK', 'DREB_RANK', 'DREB_PCT_RANK',
        'PCT_DREB_RANK', 'STL_RANK', 'PCT_STL_RANK', 'BLK_RANK', 'PCT_BLK_RANK', 'OPP_PTS_OFF_TOV_RANK',
        'OPP_PTS_2ND_CHANCE_RANK', 'OPP_PTS_FB_RANK', 'OPP_PTS_PAINT_RANK', 'DEF_WS_RANK', 'CFID', 'CFPARAMS'], inplace=True)
diff = list(set(df['PLAYER_ID'].tolist()) - set(def_data['PLAYER_ID'].tolist()))
df = df[~df['PLAYER_ID'].isin(diff)]

df.sort_values(by='PLAYER_NAME', inplace=True)

# Insert rating data
def_data = def_data.join(df[df.columns[2:]])

def_data.shape

(484, 33)

In [62]:
def_data.to_csv(loc + season + '_def_stats.csv', index=False)